#### Other Declarative Mapping Directives

* `__declare_last__()`: The `__declare_last__()` _hook_ __allows definition of a class level function__ that is _automatically called_ by the `MapperEvents.after_configured()` event, which occurs __after mappings are assumed to be completed__ and the __`"configure"` step has finished__.

In [1]:
from sqlalchemy import Column, Integer, String, Table, MetaData, PrimaryKeyConstraint
from sqlalchemy.orm import declared_attr, registry, declarative_base

In [2]:
Base = declarative_base()

In [3]:
class AfterUserClass(Base):
    __tablename__ = "after_user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    firstname = Column(String(50))
    lastname = Column(String(50))
    
    @classmethod
    def __declare_last__(cls):
        print("configured already")

* `__declare_first__()`: Like `__declare_last__()`, but is called at the __beginning of mapper configuration__ via the `MapperEvents.before_configured()` event.

In [4]:
class BeforeUserClass(Base):
    __tablename__ = "before_user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    firstname = Column(String(50))
    lastname = Column(String(50))
    
    @classmethod
    def __declare_first__(cls):
        print("initializing configuration")

* `metadata`: The `MetaData` collection _normally used to assign_ a __new `Table`__ is the `registry.metadata` attribute _associated with_ the `registry` object in use. When using a _declarative base class_ such as that __generated__ by `declarative_base()` as well as `registry.generate_base()`, this `MetaData` is also _normally present_ also as an _attribute_ named `.metadata` that's __directly on the base class__, and thus also on the __mapped class via inheritance__. _Declarative uses this attribute_, when present, in order to __determine the target `MetaData` collection__, or if _not present_, uses the `MetaData` __associated directly with the registry__.

This _attribute_ may also be _assigned towards_ in order to __affect the `MetaData` collection__ to be used on a __per-mapped-hierarchy basis for a single base and/or registry__. This _takes effect whether a declarative base class is used_ or if the `registry.mapped()` _decorator is used directly_, thus _allowing patterns_ such as the `metadata-per-abstract base` example in the next section, `__abstract__`. A _similar pattern_ can be illustrated using `registry.mapped()` as follows.

In [5]:
reg = registry()

In [6]:
class BaseOne:
    metadata = MetaData()

In [7]:
class BaseTwo:
    metadata = MetaData()

In [8]:
@reg.mapped
class ClassOne:
    __tablename__ = "t1"  # will use reg.metadata
    
    id = Column(Integer, primary_key=True)

In [9]:
@reg.mapped
class ClassTwo(BaseOne):
    __tablename__ = "t2"  # will use BaseOne.metadata
    
    id = Column(Integer, primary_key=True)

In [10]:
@reg.mapped
class ClassThree(BaseTwo):
    __tablename__ = "t3"  # will use BaseTwo.metadata
    
    id = Column(Integer, primary_key=True)

* `__abstract__`: `__abstract__` causes declarative to _skip the production of a table or mapper_ for the class __entirely__. A _class can be added within a hierarchy_ in the same way as `mixin` (see `Mixin and Custom Base Classes`), __allowing subclasses to extend just from the special class__.

In [11]:
class AbstractUserClass(Base):
    __astract__ = True
    __tablename__ = "abstract_user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    firstname = Column(String(50))
    lastname = Column(String(50))
    
    def helpful_abstract_method(self):
        print("inside abstract method")
    
    @declared_attr
    def __mapper_args__(cls):
        return {
            "exclude_properties": [
                column.key for column in cls.__table__.c
                if column.info.get("exclude", False)
            ]
        }

In [12]:
class MappedUserClass(AbstractUserClass):
    pass

_One possible use_ of `__abstract__` is to __use a distinct `MetaData` for different bases__.

In [13]:
class DefaultBase(Base):
    __abstract__ = True
    metadata = MetaData()

In [14]:
class OtherBase(Base):
    __abstract__ = True
    metadata = MetaData()

Above, _classes which inherit from_ `DefaultBase` will use one `MetaData` as the __registry of tables__, and _those which inherit from_ `OtherBase` will __use a different one__. The _tables themselves_ can then be __created perhaps within distinct databases__.

```
DefaultBase.metadata.create_all(engine_one)
OtherBase.metadata.create_all(engine_two)
```

* `__table_cls__`: Allows the _callable/class_ used to _generate_ a `Table` to be __customized__. This is a very `open-ended hook` that can __allow special customizations__ to a `Table` that one generates here.

In [15]:
class CustomMixin(object):
    @classmethod
    def __table_cls__(cls, name, metadata_obj, *args, **kwargs):
        return Table(f"custom_{name}", metadata_obj, *args, **kwargs)

The above _mixin_ would cause _all_ `Table` _objects_ generated to __include the prefix `"custom_"`, followed by the name__ normally specified using the `__tablename__` attribute.

`__table_cls__` also _supports_ the case of _returning_ `None`, which _causes the class_ to be __considered as single-table inheritance vs. its subclass__. This _may be useful in some customization schemes_ to determine that `single-table inheritance` should take place based on the _arguments for the table itself_, such as, define as _single-inheritance_ if there is __no primary key present__.

In [16]:
class AutoTable(object):
    @declared_attr
    def __tablename__(cls):
        return cls.__name__
    
    @classmethod
    def __table_cls__(cls, *args, **kwargs):
        for obj in args[1:]:
            if (isinstance(obj, Column) and obj.primary_key) or isinstance(obj, PrimaryKeyConstraint):
                return Table(*args, **kwargs)
        return None

In [17]:
class Person(AutoTable, Base):
    id = Column(Integer, primary_key=True)

In [18]:
class Employee(Person):
    employee_name = Column(String)

The above `Employee` class would be __mapped as single-table inheritance against__ `Person`; the `employee_name` column would be _added as a member_ of the `Person` table.